## DEMO: Edit Distance Performance

In [ ]:
%pip install pandas thefuzz rapidfuzz

In [ ]:
import random

import pandas as pd

from thefuzz import fuzz

from rapidfuzz import fuzz as rapid_fuzz
from rapidfuzz.process import cdist
from rapidfuzz.distance.OSA import normalized_distance


In [ ]:
def generate_ids():
    """Generate ID-like strings."""
    while True:
        country_code = "NLD"
        gender = random.choice(["M", "F"])
        first = f"{random.randint(0, 9999999):07d}"
        second = f"{random.randint(0, 9999999999999999):016d}"

        yield f"{country_code}{first}{gender}{second}"


def similarity(target_id: str, search_ids: pd.Series) -> pd.Series:
    """Compute Levenhstein distance between IDs."""
    return search_ids.map(lambda search_id: fuzz.ratio(target_id, search_id))


def rapid_similarity(target_id: str, search_ids: pd.Series) -> pd.Series:
    """Compute Levenhstein distance between IDs."""
    return pd.Series(
        cdist([target_id], search_ids, scorer=rapid_fuzz.ratio, workers=-1)[0]
    )


def rapid_distance(target_id: str, search_ids: pd.Series) -> pd.Series:
    """Compute Optimal String Alignment distance between IDs."""
    return pd.Series(
        cdist([target_id], search_ids, scorer=normalized_distance, workers=-1)[0]
    )

def equality(target_id: str, search_ids: pd.Series) -> pd.Series:
    """Compare an ID to a list of IDs."""
    return search_ids.map(lambda search_id: search_id == target_id)

In [ ]:
id_generator = generate_ids()

In [ ]:
n = 5_000_000
ids = pd.Series([next(id_generator) for _ in range(n)])
ids.sample(5)

In [ ]:
top_n = 10

In [ ]:
similarities = rapid_similarity(ids[0], ids).sort_values(ascending=False)
similarities.head(top_n)

In [ ]:
ids.iloc[similarities.index[0:top_n]]

In [ ]:
distances = rapid_distance(ids[0], ids).sort_values(ascending=True)
distances.head(top_n)

In [ ]:
ids.iloc[distances.index[0:top_n]]

In [ ]:
%timeit similarity(ids[0], ids)

In [ ]:
%timeit rapid_similarity(ids[0], ids)

In [ ]:
%timeit rapid_distance(ids[0], ids)

In [ ]:
%timeit equality(ids[0], ids)